In [1]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

%load_ext yamlmagic

%load_ext autoreload
%autoreload 

In [2]:
%%yaml wmh_inference_config

log:
    root: '?'
    log_interval: 50
    track_image_level: True
    track_pixel_level: True 
    save_calbration_error_maps: False 

model:
    exp_root : /storage/vbutoi/scratch/ESE/training/10_23_23_Dense_WMH
    num_workers: 1  

dataset:
    split: '?' 
    task: Amsterdam 
    slicing: dense_full 
    input_type: volume 

calibration:
    bin_weightings:
        - proportional
    conf_interval_start: 0.5
    conf_interval_end: 1.0
    num_bins: 10

score:
    ignore_index: 0

<IPython.core.display.Javascript object>

In [3]:
%%yaml cal_metrics_cfg 

cal_metrics:
    - ECE:
        func: ese.experiment.metrics.ECE
    - TL_ECE:
        func: ese.experiment.metrics.TL_ECE
    - CW_ECE:
        func: ese.experiment.metrics.CW_ECE
    - SUME:
        func: ese.experiment.metrics.SUME
    - TL_SUME:
        func: ese.experiment.metrics.TL_SUME
    - CW_SUME:
        func: ese.experiment.metrics.CW_SUME

<IPython.core.display.Javascript object>

In [4]:
from ionpy.util import dict_product, Config

# Need to define the experiment name
exp_name = "11_16_23_WMH_Label0_Cal"

# Get the options for WMH
##################################################
wmh_dataset_options = {
    'log.root': [f'/storage/vbutoi/scratch/ESE/inference/{exp_name}'],
    'dataset.split': ['cal'],
}

base_wmh_cfg = Config(wmh_inference_config).update(cal_metrics_cfg)

wmh_cfgs = []
for cfg_update in dict_product(wmh_dataset_options):
    new_wmh_cfg = base_wmh_cfg.update(cfg_update)
    wmh_cfgs.append(new_wmh_cfg)

In [5]:
from ese.experiment.analysis.inference import get_cal_stats
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

get_cal_stats(wmh_cfgs[0])

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")


KeyError: 'ignore_index'

In [ ]:
# from ionpy.slite.submit import submit_jobs
# from ese.experiment.analysis.inference import get_cal_stats

# submit_jobs(
#     project="ESE",
#     exp_name=exp_name,
#     job_mode="inference",
#     config_list=wmh_cfgs, 
#     job_func=get_cal_stats,
#     available_gpus=["0"]
#     )